In [8]:
import keras_tuner
from tensorflow import keras
import keras_tuner
from keras_tuner.tuners import RandomSearch
import pandas as pd

In [2]:
adv_df = pd.read_csv("./data/shadow_model_outputs.csv")
adv_ddf_shuffled = adv_df.sample(frac=1, random_state=1).reset_index(drop=True).drop(columns=["Unnamed: 0"])
adv_y = adv_ddf_shuffled["y"]
adv_X = adv_ddf_shuffled.drop(columns=["y"])

In [3]:
adv_df_test = pd.read_csv("./data/shadow_model_outputs_test_set.csv")
adv_ddf_shuffled_test = adv_df_test.sample(frac=1, random_state=1).reset_index(drop=True).drop(columns=["Unnamed: 0"])
adv_y_test = adv_ddf_shuffled_test["y"]
adv_X_test = adv_ddf_shuffled_test.drop(columns=["y"])

In [4]:
def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.Dense(units=hp.Int('units', min_value=8, max_value=136, step=8)))
    model.add(keras.layers.Dense(units=hp.Int('units', min_value=4, max_value=32, step=4)))
    model.add(keras.layers.Dense(1))
    
    model.compile(optimizer='adam', loss=keras.losses.MeanSquaredError(), metrics=keras.metrics.R2Score())
    return model

In [35]:
tuner = RandomSearch(
    build_model,
    #objective=keras_tuner.Objective('val_r2_score', direction='max'),
    objective = 'val_loss',
    max_trials = 2,
    executions_per_trial = 3,
)

Reloading Tuner from ./untitled_project/tuner0.json


In [36]:
tuner.search(adv_X, adv_y, epochs=15, validation_data=(adv_X_test, adv_y_test))

In [37]:
models = tuner.get_best_models(num_models=2)

In [38]:
best_model = models[0]

In [39]:
best_model.build(input_shape=(None, 10001,))

In [ ]:
#X_train, y_train, X_test, y_test, _, _ = data_train_test(train_size=0.7)